##Final Code

In [1]:
!pip install fasttext
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199771 sha256=69427ff1713394be3d71c01b12696daf11b83b6c04841c988e65788b9196a2c1
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [2]:
import fasttext.util
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import re
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
data1 = pd.read_csv("/content/drive/MyDrive/Healthcare/SmartDocPad/text_classification/data/UCI_ML_Drug_Review/drugsComTrain_raw.csv")
data2 = pd.read_csv("/content/drive/MyDrive/Healthcare/SmartDocPad/text_classification/data/UCI_ML_Drug_Review/drugsComTest_raw.csv")
data = pd.concat([data1,data2])[['review','condition']]

In [4]:
data.to_csv("/content/drive/MyDrive/Healthcare/SmartDocPad/text_classification/data/UCI_ML_Drug_Review/drugsReview.csv",index=False)

In [5]:
count_df = data[['condition','review']].groupby('condition').aggregate({'review':'count'}).reset_index().sort_values('review',ascending=False)
target_conditions = count_df[count_df['review']>3000]['condition'].values
target_conditions

array(['Birth Control', 'Depression', 'Pain', 'Anxiety', 'Acne',
       'Bipolar Disorde', 'Insomnia', 'Weight Loss', 'Obesity', 'ADHD',
       'Diabetes, Type 2', 'Emergency Contraception',
       'High Blood Pressure', 'Vaginal Yeast Infection'], dtype=object)

In [6]:
def condition_parser(x):
    if x in target_conditions:
        return x
    else:
        return "OTHER"

def clean_text(x):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', x)
    return x

def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

stop_words = set(stopwords.words('english'))

def preprocess_sentence(text):
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()

    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]

    return ' '.join(tokens)


In [7]:
def pre_process(data, min_label_samples=3000):
  df = data.copy()
  # remove NULL Values from data
  df = df[pd.notnull(df['review'])]

  df['condition'] = df['condition'].apply(lambda x: condition_parser(x))

  df = df[df['condition']!='OTHER']

  # lower the text
  df["review"] = df["review"].apply(lambda x: x.lower())

  # Clean the text
  df["review"] = df["review"].apply(lambda x: clean_text(x))

  # Clean numbers
  df["review"] = df["review"].apply(lambda x: clean_numbers(x))

  # Clean Contractions
  df["review"] = df["review"].apply(lambda x: replace_contractions(x))

  df['tokenize'] = df['review'].apply(preprocess_sentence)

  df = df.reset_index(drop=True)

  return df

In [8]:
data = pre_process(data)

In [9]:
label_encoder = LabelEncoder()
data['condition_numerical'] = label_encoder.fit_transform(data['condition'])

In [10]:
data

,review,condition,tokenize,condition_numerical
0,"""my son is halfway through his fourth week of ...",ADHD,`` son halfway fourth week intuniv became conc...,0
1,"""i used to take another oral contraceptive, wh...",Birth Control,`` used take another oral contraceptive pill c...,4
2,"""this is my first time using any form of birth...",Birth Control,`` first time using form birth control glad we...,4
3,"""he pulled out, but he cummed a bit in me. i t...",Emergency Contraception,`` pulled cummed bit took plan b hours later t...,7
4,"""abilify changed my life. there is hope. i was...",Bipolar Disorde,`` abilify changed life hope zoloft clonidine ...,3
...,...,...,...,...
111559,"""i was on microgestin for about 3 years. over ...",Birth Control,`` microgestin 3 years past year started get e...,4
111560,"""i started taking apri about 7 months ago. my ...",Birth Control,`` started taking apri 7 months ago breats got...,4
111561,"""i&####;ve been taking lexapro (escitaploprgra...",Anxiety,`` taking lexapro escitaploprgram since februa...,2
111562,"""i&####;m married, ## years old and i have no ...",Birth Control,`` married years old kids taking pill hassle d...,4


In [11]:
import plotly.express as px
count_df = data[['condition','review']].groupby('condition').aggregate({'review':'count'}).reset_index().sort_values('review',ascending=False)
px.bar(count_df[count_df['review']>3000],x='condition',y='review')

In [12]:
print(data["review"][1])

"i used to take another oral contraceptive, which had ## pill cycle, and was very happy- very light periods, max 5 days, no other side effects. but it contained hormone gestodene, which is not available in us, so i switched to lybrel, because the ingredients are similar. when my other pills ended, i started lybrel immediately, on my first day of period, as the instructions said. and the period lasted for two weeks. when taking the second pack- same two weeks. and now, with third pack things got even worse- my third period lasted for two weeks and now it&####;s the end of the third week- i still have daily brown discharge.
the positive side is that i didn&####;t have any other side effects. the idea of being period free was so tempting... alas."


In [13]:
# Define the TF-IDF vectorizer with desired parameters
vectorizer = TfidfVectorizer(max_features=2000) #TfidfVectorizer(stop_words='english', min_df=0, max_df=1, use_idf=True, ngram_range = (1,3))

# Fit the vectorizer on the encoded category data
tfidf_review = vectorizer.fit_transform(data['tokenize'])

# Convert the sparse matrix to a dense array
tfidf_review = tfidf_review.toarray()

# Print the shape of the feature vectors
print(tfidf_review.shape)

(111564, 2000)


In [14]:
import pickle
with open('/content/drive/MyDrive/Healthcare/SmartDocPad/text_classification/data/UCI_ML_Drug_Review/saved_models/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)


In [17]:
tfidf_review[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [16]:
# Separate features and target
X = tfidf_review
y = data['condition_numerical']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

In [19]:
# Configure the XGBClassifier for GPU usage
param = {'nthread': 6}
#param['gpu_id'] = 0
param['tree_method'] = 'gpu_hist' #'hist' #'gpu_hist'
param['device'] = 'cuda' #'cpu' #'cuda'

# Train the XGBClassifier
xgb_model = xgb.XGBClassifier(objective="multi:softprob", num_class=y_train.max()+1, **param).fit(X_train, y_train)

# Make predictions on the test data
predictions = xgb_model.predict(X_test)

# Evaluate the model
f1 = f1_score(y_test, predictions, average='macro')
accuracy = accuracy_score(y_test, predictions)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print("F1 Score:", f1)
print("Accuracy Score:", accuracy)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[16:19:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[16:20:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[16:20:30] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the devi

              precision    recall  f1-score   support

           0       0.96      0.86      0.91       452
           1       0.96      0.91      0.93       727
           2       0.80      0.77      0.79       758
           3       0.85      0.75      0.80       528
           4       0.96      0.99      0.97      3894
           5       0.74      0.86      0.80      1243
           6       0.94      0.86      0.90       323
           7       0.99      0.97      0.98       326
           8       0.91      0.78      0.84       321
           9       0.91      0.85      0.87       513
          10       0.80      0.78      0.79       454
          11       0.90      0.93      0.91       811
          12       0.97      0.94      0.96       329
          13       0.82      0.79      0.81       478

    accuracy                           0.90     11157
   macro avg       0.89      0.86      0.88     11157
weighted avg       0.90      0.90      0.90     11157

[[ 390    1    5    4    

In [20]:
# Save the model
with open('/content/drive/MyDrive/Healthcare/SmartDocPad/text_classification/data/UCI_ML_Drug_Review/saved_models/drug_review_tfidf_classifier.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)

In [21]:
# Load the saved model
with open('/content/drive/MyDrive/Healthcare/SmartDocPad/text_classification/data/UCI_ML_Drug_Review/saved_models/drug_review_tfidf_classifier.pkl', 'rb') as f:
    xgb_model = pickle.load(f)

In [22]:
# Function for preprocessing of text
def preprocess_text(text):
    text = preprocess_sentence(text)
    text = clean_text(text)
    text = clean_numbers(text)
    text = replace_contractions(text)
    return text

In [23]:
def predict_condition_label(text):
  processed_text = preprocess_text(text)
  review = vectorizer.transform([processed_text])
  pred = xgb_model.predict(review.toarray())

  return label_encoder.classes_[pred[0]]

In [24]:
data["review"][1]

'"i used to take another oral contraceptive, which had ## pill cycle, and was very happy- very light periods, max 5 days, no other side effects. but it contained hormone gestodene, which is not available in us, so i switched to lybrel, because the ingredients are similar. when my other pills ended, i started lybrel immediately, on my first day of period, as the instructions said. and the period lasted for two weeks. when taking the second pack- same two weeks. and now, with third pack things got even worse- my third period lasted for two weeks and now it&####;s the end of the third week- i still have daily brown discharge.\r\nthe positive side is that i didn&####;t have any other side effects. the idea of being period free was so tempting... alas."'

In [25]:
data["condition"][1]

'Birth Control'

In [26]:
label_encoder.classes_

array(['ADHD', 'Acne', 'Anxiety', 'Bipolar Disorde', 'Birth Control',
       'Depression', 'Diabetes, Type 2', 'Emergency Contraception',
       'High Blood Pressure', 'Insomnia', 'Obesity', 'Pain',
       'Vaginal Yeast Infection', 'Weight Loss'], dtype=object)

In [27]:
predict_condition_label(data["review"][1])

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[16:20:39] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"




'Birth Control'

In [28]:
!pip -q install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.6 MB/s eta 0:00:00


In [29]:
import gradio

In [30]:
# Input from user
in_review = gradio.Textbox(lines=20, placeholder=None, value="review", label='Enter Review Text')

In [31]:
# Output prediction
out_label = gradio.Textbox(type="text", label='Predicted Condition Label')

In [32]:
# Gradio interface to generate UI link

iface = gradio.Interface(fn = predict_condition_label,
                         inputs = [in_review],
                         outputs = [out_label])

iface.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://782015c6aa34f18850.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
